# Solution 3

In [1]:
# !pip install gcastle

In [2]:
# Basic information of alarms
# viz the graph

# Try TTPM Method
# Try notears
# Try another method in book


# S2: try more preprocessing

In [3]:
# S_1_2 notears

In [4]:
import os
import numpy as np
import pandas as pd

import os
from castle.common.priori_knowledge import PrioriKnowledge
# os.path.abspath('.')

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [5]:
import plotly.express as px
import plotly.graph_objects as go


In [6]:
from methods import *

2023-10-01 18:00:41,079 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/backend/__init__.py[line:36] - INFO: You can use `os.environ['CASTLE_BACKEND'] = backend` to set the backend(`pytorch` or `mindspore`).
2023-10-01 18:00:41,292 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/__init__.py[line:36] - INFO: You are using ``pytorch`` as the backend.


# Read Data

In [7]:
dataset = 'dataset_1'

def read_data(dataname = 'dataset_1'):
    # alarm data
    alarms = pd.read_csv(r'./datasets/dataset_1/alarm.csv')
    # causal_prior
    causal_prior= np.load(r'./datasets/dataset_1/causal_prior.npy')

    # topology
    topology = np.load(r'./datasets/dataset_1/topology.npy')

    # rca
    rca_prior = pd.read_csv(r'./datasets/dataset_1/rca_prior.csv')

    print(f"shape of alarm data: {alarms.shape}")
    print(f"shape of causal prior matrix: {causal_prior.shape}")
    print(f"shape of topology prior matrix: {topology.shape}")
    print(f"shape of rca prior matrix: {rca_prior.shape}")
    # Notes: topology.npy and rca_prior.csv are not used in this script.
    
    return alarms, causal_prior, topology, rca_prior

In [8]:
# dataset = 'dataset_4'

def read_data(dataset = 'dataset_1'):

    # alarm data
    alarms = pd.read_csv(r'./datasets/{}/alarm.csv'.format(dataset))
    # causal_prior
    causal_prior= np.load(r'./datasets/{}/causal_prior.npy'.format(dataset))

    # # topology
    topology = np.load(r'./datasets/{}/topology.npy'.format(dataset))

    # # rca
    rca_prior = pd.read_csv(r'./datasets/{}/rca_prior.csv'.format(dataset))

    print(f"shape of alarm data: {alarms.shape}")
    print(f"shape of causal prior matrix: {causal_prior.shape}")
    print(f"shape of topology prior matrix: {topology.shape}")
    print(f"shape of rca prior matrix: {rca_prior.shape}")
    print("_________")
    
    prior_knowledge = PrioriKnowledge(causal_prior.shape[0])
    for i, j in zip(*np.where(causal_prior == 1)):
        prior_knowledge.add_required_edge(i, j)

    for i, j in zip(*np.where(causal_prior == 0)):
        prior_knowledge.add_forbidden_edge(i, j)
    
    return alarms, causal_prior, topology, rca_prior, prior_knowledge


# Notes: topology.npy and rca_prior.csv are not used in this script.

In [9]:
list_of_dataset =[]
for dataset in ['dataset_1', 'dataset_2', 'dataset_3']:
    alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = dataset)
    # print(f"dataset: {dataset}")
    # print(f"shape of alarm data: {alarms.shape}")
    # print(f"shape of causal prior matrix: {causal_prior.shape}")
    # print(f"shape of topology prior matrix: {topology.shape}")
    # print(f"shape of rca prior matrix: {rca_prior.shape}")
    # print("_________")
    
    list_of_dataset.append([dataset,alarms, causal_prior, topology, rca_prior, prior_knowledge])
    
# alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = 'dataset_1')
# alarms, causal_prior, topology, rca_prior, prior_knowledge = read_data(dataset = 'dataset_2')
# alarms, causal_prior, topology, rca_prior, prior_knowledge= read_data(dataset = 'dataset_3')
# alarms, causal_prior, topology, rca_prior = read_data(dataname = 'dataset_1')

shape of alarm data: (141853, 4)
shape of causal prior matrix: (39, 39)
shape of topology prior matrix: (35, 35)
shape of rca prior matrix: (49, 2)
_________
shape of alarm data: (132688, 4)
shape of causal prior matrix: (49, 49)
shape of topology prior matrix: (42, 42)
shape of rca prior matrix: (47, 2)
_________
shape of alarm data: (198962, 4)
shape of causal prior matrix: (31, 31)
shape of topology prior matrix: (39, 39)
shape of rca prior matrix: (43, 2)
_________


In [10]:
list_of_dataset

[['dataset_1',
          alarm_id  device_id  start_timestamp  end_timestamp
  0             12         25                5             38
  1             10         25                7             45
  2              2         24               13             53
  3             35         17               13             86
  4             35         32               13             14
  ...          ...        ...              ...            ...
  141848        25         21           604772         604883
  141849        25          2           604779         604863
  141850        25         26           604787         604827
  141851        27          0           604787         604881
  141852        27         26           604816         604856
  
  [141853 rows x 4 columns],
  array([[-1., -1., -1., ...,  0., -1., -1.],
         [-1., -1., -1., ...,  0., -1., -1.],
         [-1., -1., -1., ...,  0., -1., -1.],
         ...,
         [ 0.,  0.,  0., ...,  0.,  0.,  0.],
         [-

# S3 的预处理

In [11]:
# device 和device的关系
topology

array([[0, 0, 0, ..., 0, 1, 1],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0]])

In [12]:
topology.shape

(39, 39)

In [13]:
import plotly.express as px
def show_matrix(matrix):
    df = pd.DataFrame(matrix).reset_index().melt(id_vars='index')
    df.columns = ['from', 'to', 'value']
    shape = matrix.shape[0]
    return px.density_heatmap(df, x='from', y='to',z='value',histfunc='avg', nbinsx=shape, nbinsy=shape,height=600, width=600)
# show_matrix(topology)

In [14]:
# 和1链接的有多少，从1出发是否能构建整个网络，层数为n

# 1. 从1出发，能到达的节点
# 2. 从1出发，能到达的节点，能到达的节点



In [15]:
alarms

,alarm_id,device_id,start_timestamp,end_timestamp
0,0,7,14,85
1,6,30,17,100
2,25,3,18,54
3,13,3,19,64
4,26,31,19,62
...,...,...,...,...
198957,13,23,604849,604889
198958,13,23,604850,604892
198959,13,27,604853,604898
198960,13,27,604896,604937


In [16]:
samples = make_window_sample(alarms, 1000)

Window size 1000 Total 198962 removed 43912 alarms


In [17]:
samples

alarm_id,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,30
win_id,,,,,,,,,,,,,,,,,,,,,
0,22.0,1.0,0.0,4.0,0.0,0.0,27.0,19.0,30.0,0.0,...,0.0,2.0,6.0,0.0,3.0,16.0,6.0,21.0,0.0,1.0
1,10.0,3.0,2.0,2.0,0.0,1.0,6.0,18.0,19.0,0.0,...,0.0,0.0,3.0,1.0,0.0,18.0,6.0,8.0,1.0,7.0
2,31.0,3.0,0.0,3.0,1.0,0.0,10.0,15.0,15.0,1.0,...,1.0,2.0,4.0,0.0,1.0,48.0,5.0,7.0,0.0,0.0
4,9.0,2.0,1.0,0.0,0.0,1.0,5.0,11.0,9.0,1.0,...,4.0,1.0,1.0,0.0,1.0,8.0,14.0,4.0,1.0,0.0
5,36.0,2.0,1.0,23.0,0.0,1.0,6.0,32.0,22.0,4.0,...,10.0,5.0,5.0,0.0,6.0,19.0,8.0,19.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599,17.0,0.0,1.0,9.0,0.0,1.0,3.0,14.0,7.0,0.0,...,2.0,2.0,7.0,2.0,4.0,14.0,3.0,3.0,0.0,1.0
600,48.0,1.0,1.0,19.0,1.0,0.0,21.0,43.0,30.0,2.0,...,8.0,3.0,13.0,1.0,6.0,24.0,15.0,32.0,0.0,4.0
601,37.0,10.0,3.0,16.0,0.0,0.0,22.0,43.0,35.0,4.0,...,5.0,4.0,18.0,0.0,11.0,42.0,20.0,23.0,2.0,2.0


In [18]:
alarms.groupby('device_id').count()

,alarm_id,start_timestamp,end_timestamp
device_id,,,
0,2040,2040,2040
1,2764,2764,2764
2,2395,2395,2395
3,11081,11081,11081
4,1131,1131,1131
5,13054,13054,13054
6,15649,15649,15649
7,10250,10250,10250
8,594,594,594


In [19]:
# ttpm.learn(samples)

# 建模

In [20]:
import os
os.environ['CASTLE_BACKEND'] = 'pytorch'

In [21]:
import importlib

import methods
import preprocess
importlib.reload(methods)
importlib.reload(preprocess)
from methods import Causal
from methods import causal_all
import torch
torch.device('mps')

device(type='mps')

In [22]:
causal_all(list_of_dataset = list_of_dataset)

Start processing dataset dataset_1 with method golemStart processing dataset dataset_2 with method golem
Processing dataset dataset_2

Processing dataset dataset_1
Start processing dataset dataset_3 with method golem
Processing dataset dataset_3
Window size 450 Total 132688 removed 24778 alarms
Window size 450 Total 198962 removed 53365 alarms
Window size 450 Total 141853 removed 31345 alarms
Window size 900 Total 141853 removed 26960 alarms
Window size 900 Total 198962 removed 44720 alarms
Window size 900 Total 132688 removed 22019 alarms
Window size 1500 Total 141853 removed 23505 alarms


2023-10-01 18:00:42,638 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:122] - INFO: GPU is unavailable.
2023-10-01 18:00:42,661 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:207] - INFO: Started training for 10000 iterations.
2023-10-01 18:00:42,666 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:122] - INFO: GPU is unavailable.
2023-10-01 18:00:42,673 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:207] - INFO: Started training for 10000 iterations.
2023-10-01 18:00:42,679 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:122] - INFO: GPU is unavailable.
2023-10-01 18:00:42,682 - /Users

Start fitting dataset_1 Shape of samples: (2176, 39)
Window size 1500 Total 198962 removed 39070 alarms
Window size 1500 Total 132688 removed 19418 alarms
Start fitting dataset_3 Shape of samples: (2177, 31)
Start fitting dataset_2 Shape of samples: (2173, 49)


2023-10-01 18:05:05,916 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:220] - INFO: [Iter 5000] score=199.936, likelihood=196.567, h=9.4e-02
2023-10-01 18:06:52,380 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:220] - INFO: [Iter 5000] score=244.860, likelihood=241.767, h=4.3e-02
2023-10-01 18:07:18,362 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:220] - INFO: [Iter 5000] score=309.413, likelihood=305.910, h=2.2e-02
2023-10-01 18:09:19,995 - /Users/zhitaogao/opt/anaconda3/envs/gcastle/lib/python3.9/site-packages/castle/algorithms/gradient/notears/torch/golem.py[line:220] - INFO: [Iter 10000] score=198.823, likelihood=196.141, h=7.3e-02


shape:  (31, 31)
Saved solution 20231001-180920-2 for dataset dataset_3 to ./submission/solution20231001-180920-2/dataset_3_graph_matrix.npy
Saved solution 2 for dataset 1 to ./submission/solution2/1_graph_matrix.npy
All dataset processed


# Evaluation

In [23]:
from castle.common import GraphDAG

In [34]:
# read solution

s_name = 'solution20231001-180920-2'

s_name_baseline = 'solution2'

In [35]:
def show_result(s_name, s_name_baseline = None):
    
    
    
    graph_matrix_1 = np.load(r'./submission/{}/dataset_1_graph_matrix.npy'.format(s_name))
    graph_matrix_2 = np.load(r'./submission/{}/dataset_2_graph_matrix.npy'.format(s_name))
    graph_matrix_3 = np.load(r'./submission/{}/dataset_3_graph_matrix.npy'.format(s_name))
    graph_matrix_4 = np.load(r'./submission/{}/dataset_4_graph_matrix.npy'.format(s_name))
    
    glist = [graph_matrix_1, graph_matrix_2, graph_matrix_3, graph_matrix_4]
    

    if s_name_baseline:
        compare_graph_matrix_1 = np.load(r'./submission/{}/dataset_1_graph_matrix.npy'.format(s_name_baseline))
        compare_graph_matrix_2 = np.load(r'./submission/{}/dataset_2_graph_matrix.npy'.format(s_name_baseline))
        compare_graph_matrix_3 = np.load(r'./submission/{}/dataset_3_graph_matrix.npy'.format(s_name_baseline))
        compare_graph_matrix_4 = np.load(r'./submission/{}/dataset_4_graph_matrix.npy'.format(s_name_baseline))
        compare_list = [compare_graph_matrix_1, compare_graph_matrix_2, compare_graph_matrix_3, compare_graph_matrix_4]
    else:
        compare_list = None
    
    from plotly.subplots import make_subplots
    # make plotly subplot
    
    fig = make_subplots(rows=1, cols=4, subplot_titles=("dataset_1", "dataset_2", "dataset_3",'dataset_4'))
    
    if compare_list:
        fig = make_subplots(rows=2, cols=4, subplot_titles=("dataset_1", "dataset_2", "dataset_3",'dataset_4', \
                                                            'baseline_1', 'baseline_2', 'baseline_3','baseline_4'))
        
    for idx, g in enumerate(glist):
        df = pd.DataFrame(g)
        df.reset_index(inplace=True)    
        df = df.melt(id_vars='index')
        # change value to category
        df['value'] = df['value'].astype('category')
        
        

        fig.add_trace(
            go.Heatmap(
                x=df['index'],
                y=df['variable'],
                z=df['value'],
                colorscale='Viridis',
                colorbar=dict(title='value'),
            ),
            row=1, col=idx+1
        )
        
    if compare_list:
        for idx, g in enumerate(compare_list):
            df = pd.DataFrame(g)
            df.reset_index(inplace=True)    
            df = df.melt(id_vars='index')
            # change value to category
            df['value'] = df['value'].astype('category')



            fig.add_trace(
                go.Heatmap(
                    x=df['index'],
                    y=df['variable'],
                    z=df['value'],
                    colorscale='Viridis',
                    colorbar=dict(title='value'),
                ),
                row=2, col=idx+1
            )
            
    
    if compare_list:
        
        for idx in range(len(glist)):
            diff = np.sum(glist[idx] != compare_list[idx])
            print(f"diff of dataset {idx}: {diff}")
        
    return fig

In [36]:
show_result(s_name, s_name_baseline = s_name_baseline)

diff of dataset 0: 96
diff of dataset 1: 102
diff of dataset 2: 71
diff of dataset 3: 0
